## Creando la sesión

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
val spark = SparkSession.builder.appName("SparkSQLExampleApp").enableHiveSupport().getOrCreate()

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@62142110


In [3]:
val csvFile="C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

csvFile: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv


In [4]:
val flightSchema = StructType(Array(StructField("date", StringType, true),
                                   StructField("delay", IntegerType, true),
                                   StructField("distance", IntegerType,true),
                                   StructField("origin", StringType, true),
                                   StructField("destination", StringType, true)))

flightSchema: org.apache.spark.sql.types.StructType = StructType(StructField(date,StringType,true), StructField(delay,IntegerType,true), StructField(distance,IntegerType,true), StructField(origin,StringType,true), StructField(destination,StringType,true))


In [5]:
val df =spark.read.format("csv")
.schema(flightSchema)
.option("header","true")
.load(csvFile)

df: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]


In [6]:
df.createOrReplaceTempView("us_delay_flights_tbl")

In [7]:
spark.sql("""SELECT *
            FROM us_delay_flights_tbl""").show(10,false)

+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
|01030605|0    |602     |ABE   |ATL        |
|01041243|10   |602     |ABE   |ATL        |
|01040605|28   |602     |ABE   |ATL        |
|01051245|88   |602     |ABE   |ATL        |
|01050605|9    |602     |ABE   |ATL        |
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [8]:
spark.sql("""SELECT distance,origin,destination 
            FROM us_delay_flights_tbl
            WHERE distance >1000
            ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [9]:
df.select("distance", "origin", "destination")
.filter($"distance">1000)
.orderBy(desc("distance"))
.show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [10]:
spark.sql("""SELECT delay,date,origin,destination 
            FROM us_delay_flights_tbl
            WHERE origin='SFO' and destination='ORD' and delay > 120 
            ORDER BY delay DESC""").show(10)

+-----+--------+------+-----------+
|delay|    date|origin|destination|
+-----+--------+------+-----------+
| 1638|02190925|   SFO|        ORD|
|  396|01031755|   SFO|        ORD|
|  326|01022330|   SFO|        ORD|
|  320|01051205|   SFO|        ORD|
|  297|01190925|   SFO|        ORD|
|  296|02171115|   SFO|        ORD|
|  279|01071040|   SFO|        ORD|
|  274|01051550|   SFO|        ORD|
|  266|03120730|   SFO|        ORD|
|  258|01261104|   SFO|        ORD|
+-----+--------+------+-----------+
only showing top 10 rows



In [11]:
df
.select("delay", "date", "origin", "destination")
.filter($"origin"==="SFO" && $"destination"==="ORD" && $"delay">120)
.orderBy(desc("delay"))
.show(10)

+-----+--------+------+-----------+
|delay|    date|origin|destination|
+-----+--------+------+-----------+
| 1638|02190925|   SFO|        ORD|
|  396|01031755|   SFO|        ORD|
|  326|01022330|   SFO|        ORD|
|  320|01051205|   SFO|        ORD|
|  297|01190925|   SFO|        ORD|
|  296|02171115|   SFO|        ORD|
|  279|01071040|   SFO|        ORD|
|  274|01051550|   SFO|        ORD|
|  266|03120730|   SFO|        ORD|
|  258|01261104|   SFO|        ORD|
+-----+--------+------+-----------+
only showing top 10 rows



In [12]:
val new_df=df.withColumn("DateTime", to_timestamp(col("date"), "MMddHHmm")).drop("date")

new_df: org.apache.spark.sql.DataFrame = [delay: int, distance: int ... 3 more fields]


In [13]:
new_df.createOrReplaceTempView("us_delay_flights_tbl")

In [14]:
spark.sql("""SELECT day(DateTime) Day,month(DateTime) Month, count(*) Count
            FROM us_delay_flights_tbl
            WHERE origin='SFO' and destination='ORD' and delay > 120 
            Group by month(DateTime), day(DateTime)
            ORDER BY count(*) DESC""").show(10)

+---+-----+-----+
|Day|Month|Count|
+---+-----+-----+
|  2|    1|    4|
|  3|    1|    4|
| 31|    3|    4|
| 12|    3|    3|
|  9|    2|    3|
|  8|    2|    2|
| 26|    3|    2|
|  5|    1|    2|
|  1|    1|    2|
| 17|    3|    2|
+---+-----+-----+
only showing top 10 rows



In [18]:
new_df
.filter($"origin"==="SFO" && $"destination"==="ORD" && $"delay">120).groupBy(month($"DateTime"),dayofmonth($"DateTime"))
.count()
.orderBy(desc("count")).show(10)

+---------------+--------------------+-----+
|month(DateTime)|dayofmonth(DateTime)|count|
+---------------+--------------------+-----+
|              1|                   2|    4|
|              1|                   3|    4|
|              3|                  31|    4|
|              3|                  12|    3|
|              2|                   9|    3|
|              2|                  13|    2|
|              1|                   5|    2|
|              2|                  27|    2|
|              1|                   1|    2|
|              3|                  17|    2|
+---------------+--------------------+-----+
only showing top 10 rows



In [174]:
spark.sql("""SELECT delay, origin, destination,
             CASE
                 WHEN delay >= 360 THEN 'Very Long Delays'
                 WHEN delay >= 120 AND delay < 360 THEN 'Long Delays'
                 WHEN delay >= 60 AND delay < 120 THEN 'Short Delays'
                 WHEN delay >= 0 AND delay < 60 THEN 'Tolerable Delays'
                 WHEN delay = 0 THEN 'No Delays'
                 ELSE 'Early'
             END AS Flight_Delays
             FROM us_delay_flights_tbl
             ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [176]:
df
.select($"delay", $"origin", $"destination",
                 when(col("delay") >= 360,"Very Long Delays")
                 .when(col("delay") >= 120 && col("delay") < 360,"Long Delays")
                 .when(col("delay") >= 60 && col("delay") < 120,"Short Delays")
                 .when(col("delay") >= 0 && col("delay") < 60,"Tolerable Delays")
                 .when(col("delay") === 0, "No Delays")
                 .otherwise("Early")
                 .alias("Flight_Delays"))
.orderBy(asc("origin"),desc("delay"))
.show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



# TABLAS MANAGED

In [10]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
val spark = SparkSession.builder.appName("SparkSQLExampleApp").enableHiveSupport().getOrCreate()

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2c7d43eb


In [11]:
spark.stop()

In [12]:
spark.sql("CREATE DATABASE learn_spark_db")

org.apache.spark.sql.AnalysisException:  org.apache.hadoop.hive.metastore.api.AlreadyExistsException: Database learn_spark_db already exists;

In [13]:
spark.sql("USE learn_spark_db")

res9: org.apache.spark.sql.DataFrame = []


In [14]:
spark.sql("""CREATE TABLE managed_us_delay_flights_tbl(date STRING, delay INT, distance INT, origin STRING, destination STRING)""")

org.apache.spark.sql.catalyst.analysis.TableAlreadyExistsException:  Table or view 'managed_us_delay_flights_tbl' already exists in database 'learn_spark_db';

In [15]:
spark.sql("show tables").show()

+--------------+--------------------+-----------+
|      database|           tableName|isTemporary|
+--------------+--------------------+-----------+
|learn_spark_db|funciona_us_delay...|      false|
|learn_spark_db|managed_us_delay_...|      false|
|learn_spark_db|prueba_us_delay_f...|      false|
|learn_spark_db|us_delay_flights_tbl|      false|
+--------------+--------------------+-----------+



In [16]:
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, distance INT, origin STRING, destination STRING)
             USING csv OPTIONS (PATH 'C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv')""")

res14: org.apache.spark.sql.DataFrame = []


In [22]:
spark.catalog.listDatabases().show()
spark.catalog.listTables().show()
spark.catalog.listColumns("us_delay_flights_tbl").show()

+--------------+----------------+--------------------+
|          name|     description|         locationUri|
+--------------+----------------+--------------------+
|       default|default database|file:/C:/Users/al...|
|learn_spark_db|                |file:/C:/Users/al...|
+--------------+----------------+--------------------+

+--------------------+--------------+-----------+---------+-----------+
|                name|      database|description|tableType|isTemporary|
+--------------------+--------------+-----------+---------+-----------+
|us_delay_flights_tbl|learn_spark_db|       null| EXTERNAL|      false|
+--------------------+--------------+-----------+---------+-----------+

+-----------+-----------+--------+--------+-----------+--------+
|       name|description|dataType|nullable|isPartition|isBucket|
+-----------+-----------+--------+--------+-----------+--------+
|       date|       null|  string|    true|      false|   false|
|      delay|       null|     int|    true|     

In [24]:
val df=spark.read.format("parquet").load("C:/Users/alvaro.romero/Big_Data/Ejercicios_Spark/Guardar_Tablas/Parquet/*")
df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Location: string (nullable =

df: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


In [25]:
val df2=spark.read.format("json").load("C:/Users/alvaro.romero/Big_Data/Ejercicios_Spark/Guardar_Tablas/JSON/*")
df2.printSchema()

root
 |-- ALSUnit: boolean (nullable = true)
 |-- Address: string (nullable = true)
 |-- AvailableDtTS: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- CallNumber: long (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- City: string (nullable = true)
 |-- FinalPriority: long (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- IncidentDate: string (nullable = true)
 |-- IncidentNumber: long (nullable = true)
 |-- Location: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- NumAlarms: long (nullable = true)
 |-- OnWatchDate: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- ResponseDelayedMins: double (nullable = true)
 |-- RowID: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Supervi

df2: org.apache.spark.sql.DataFrame = [ALSUnit: boolean, Address: string ... 26 more fields]


In [11]:
val df3=spark.read.format("csv")
.option("inferSchema", "true")
.option("header","false")
.option("mode","PERMISSIVE")
.option("separator",",")
.load("C:/Users/alvaro.romero/Big_Data/Ejercicios_Spark/Guardar_Tablas/CSV/*")
df3.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: boolean (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: timestamp (nullable = true)
 |-- _c26: timestamp (nullable = true)
 |-- _c27: timestamp

df3: org.apache.spark.sql.DataFrame = [_c0: int, _c1: string ... 26 more fields]


In [27]:
val df4=spark.read.format("avro").load("C:/Users/alvaro.romero/Big_Data/Ejercicios_Spark/Guardar_Tablas/AVRO/*")
df4.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Location: string (nullable =

df4: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


In [22]:
val parquetFile="C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"
val jsonFile="C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
val csvFile="C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
val orcFile="C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
val avroFile="C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
val schema= "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"

parquetFile: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet
jsonFile: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*
csvFile: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*
orcFile: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*
avroFile: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*
schema: String = DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, ...


In [23]:
val df = spark
.read
.format("parquet")
.option("path", parquetFile)
.load()

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [24]:
val df2 = spark.read.parquet(parquetFile)

df2: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [25]:
df.show()
df2.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [34]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
            USING parquet
            OPTIONS ( path "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet")""")

res17: org.apache.spark.sql.DataFrame = []


In [35]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [36]:
val df = spark
.read
.format("json")
.option("path", jsonFile)
.load()

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [37]:
df.show(10,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
|United States    |Singapore          |1    |
|United States    |Grenada            |62   |
|Costa Rica       |United States      |588  |
|Senegal          |United States      |40   |
|Moldova          |United States      |1    |
+-----------------+-------------------+-----+
only showing top 10 rows



In [38]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
            USING json
            OPTIONS ( path "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")""")

res20: org.apache.spark.sql.DataFrame = []


In [39]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
|United States    |Singapore          |1    |
|United States    |Grenada            |62   |
|Costa Rica       |United States      |588  |
|Senegal          |United States      |40   |
|Moldova          |United States      |1    |
+-----------------+-------------------+-----+
only showing top 10 rows



In [41]:
val df = spark
.read
.format("csv")
.option("header","true")
.schema(schema)
.option("mode","FAILFAST")
.option("nullValue","")
.option("path",csvFile)
.load()

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [42]:
df.show(10,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [43]:
val df2 = spark
  .read
  .option("header", "true")
  .option("mode", "FAILFAST") 
  .option("nullValue", "")
  .schema(schema)
  .csv(csvFile)

df2: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [44]:
df2.show(10,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [45]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING csv
    OPTIONS (
      path "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*",
      header "true",
      inferSchema "true",
      mode "FAILFAST"
    )""")

res24: org.apache.spark.sql.DataFrame = []


In [46]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [47]:
val df = spark
.read
.format("orc")
.option("path", orcFile)
.load()

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [49]:
df.show(10,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [50]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING orc
    OPTIONS (
      path "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
    )""")

res28: org.apache.spark.sql.DataFrame = []


In [51]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [52]:
val df = spark.read
  .format("avro")
  .option("path", avroFile)
  .load()

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [53]:
df.show(10,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [56]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING avro
    OPTIONS (
      path "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
    )""")

res33: org.apache.spark.sql.DataFrame = []


In [57]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [61]:
import org.apache.spark.ml.source.image

val imageDir = "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
val imagesDF = spark.read.format("image").load(imageDir)

imagesDF.printSchema
imagesDF.select("image.height", "image.width", "image.nChannels", "image.mode", "label").show(5,false)

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)
 |-- label: integer (nullable = true)

+------+-----+---------+----+-----+
|height|width|nChannels|mode|label|
+------+-----+---------+----+-----+
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |1    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
+------+-----+---------+----+-----+
only showing top 5 rows



import org.apache.spark.ml.source.image
imageDir: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/
imagesDF: org.apache.spark.sql.DataFrame = [image: struct<origin: string, height: int ... 4 more fields>, label: int]


In [64]:
val binaryFilesDF = spark.read.format("binaryFile")
.option("pathGlobFilter", "*.jpg")
.load(imageDir)

binaryFilesDF.show(5)

+--------------------+-------------------+------+--------------------+-----+
|                path|   modificationTime|length|             content|label|
+--------------------+-------------------+------+--------------------+-----+
|file:/C:/Users/al...|2021-04-15 02:34:17| 55037|[FF D8 FF E0 00 1...|    0|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54634|[FF D8 FF E0 00 1...|    1|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54624|[FF D8 FF E0 00 1...|    0|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54505|[FF D8 FF E0 00 1...|    0|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54475|[FF D8 FF E0 00 1...|    0|
+--------------------+-------------------+------+--------------------+-----+
only showing top 5 rows



binaryFilesDF: org.apache.spark.sql.DataFrame = [path: string, modificationTime: timestamp ... 3 more fields]


In [66]:
val binaryFilesDF = spark.read.format("binaryFile")
  .option("pathGlobFilter", "*.jpg")
  .option("recursiveFileLookup", "true")
  .load(imageDir)
binaryFilesDF.show(5)

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|file:/C:/Users/al...|2021-04-15 02:34:17| 55037|[FF D8 FF E0 00 1...|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54634|[FF D8 FF E0 00 1...|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54624|[FF D8 FF E0 00 1...|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54505|[FF D8 FF E0 00 1...|
|file:/C:/Users/al...|2021-04-15 02:34:17| 54475|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows



binaryFilesDF: org.apache.spark.sql.DataFrame = [path: string, modificationTime: timestamp ... 2 more fields]
